##**Building a Retrieval Augmented Generation (RAG) Chatbot**

Using Gemini, LangChain, and ChromaDB

This notebook will guide you through implementing the backend components of a RAG chatbot system.

##Setup and Prerequisites

First, let's install the necessary libraries.

In [ ]:
# Install required packages
!pip install langchain langchain-google-genai langchain_community pypdf chromadb sentence-transformers -q
!pip install google-generativeai pdfplumber -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 789.8 kB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.2 MB/s eta 0:00:00


Next, let's import all required libraries:

In [ ]:
import os
import pdfplumber
import google.generativeai as genai
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

In [ ]:
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get("GEMINI_API_KEY")

##**Section 1: Uploading PDF**
In this section, we'll implement the functionality to upload PDF files. For this notebook demonstration, we'll assume the PDF is in a local path.

In [ ]:
def upload_pdf(pdf_path):
    """
    Function to handle PDF uploads.

    Args:
        pdf_path (str): Path to the PDF file

    Returns:
        str: PDF file path if successful
    """
    try:
        # In a real application with Streamlit, you would use:
        # uploaded_file = st.file_uploader("Choose a PDF file", type="pdf")
        # But for this notebook, we'll just verify the file exists

        if os.path.exists(pdf_path):
            print(f"PDF file found at: {pdf_path}")
            return pdf_path
        else:
            print(f"Error: File not found at {pdf_path}")
            return None
    except Exception as e:
        print(f"Error uploading PDF: {e}")
        return None

In [ ]:
attention_paper_path = "/content/attention_paper.pdf"

In [ ]:
upload_pdf(attention_paper_path)

PDF file found at: /content/attention_paper.pdf


'/content/attention_paper.pdf'

##**Section 2: Parsing the PDF and Creating Text Files**
Now we'll extract the text content from the uploaded PDFs.

In [ ]:
def parse_pdf(pdf_path):
    """
    Function to extract text from PDF files.

    Args:
        pdf_path (str): Path to the PDF file

    Returns:
        str: Extracted text from the PDF
    """
    try:
        text = ""

        # Using pdfplumber to extract text
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() + "\n"

        # Save the extracted text to a file (optional)
        text_file_path = pdf_path.replace('.pdf', '.txt')
        with open(text_file_path, 'w', encoding='utf-8') as f:
            f.write(text)

        print(f"PDF parsed successfully, extracted {len(text)} characters")
        return text
    except Exception as e:
        print(f"Error parsing PDF: {e}")
        return None

In [ ]:
text_file = parse_pdf(attention_paper_path)

PDF parsed successfully, extracted 35526 characters


##**Section 3: Creating Document Chunks**
To effectively process and retrieve information, we need to break down our document into manageable chunks.

In [ ]:
def create_document_chunks(text):
    """
    Function to split the document text into smaller chunks for processing.

    Args:
        text (str): The full text from the PDF

    Returns:
        list: List of text chunks
    """
    try:
        # Initialize the text splitter
        # We can tune these parameters based on our needs and model constraints
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=500,        # Size of each chunk in characters
            chunk_overlap=100,      # Overlap between chunks to maintain context
            length_function=len,
            separators=["\n\n", "\n", " ", ""]  # Hierarchy of separators to use when splitting
        )

        # Split the text into chunks
        chunks = text_splitter.split_text(text)

        print(f"Document split into {len(chunks)} chunks")
        return chunks
    except Exception as e:
        print(f"Error creating document chunks: {e}")
        return []

In [ ]:
text_chunks = create_document_chunks(text_file)

Document split into 91 chunks


##**Section 4: Embedding the Documents**
Now we'll create vector embeddings for each text chunk using Gemini's embedding model.

In [ ]:
def embed_and_view(text_chunks):
    """
    Embed document chunks and display their numeric embeddings.

    Args:
        text_chunks (list): List of text chunks from the document
    """
    try:
        # Initialize the Gemini embeddings
        embedding_model = GoogleGenerativeAIEmbeddings(
            model="models/text-embedding-004"  # Specify the Gemini Embedding model
        )

        print("Embedding model initialized successfully")

        # Generate and display embeddings for all chunks
        for i, chunk in enumerate(text_chunks):
            embedding = embedding_model.embed_query(chunk)
            print(f"Chunk {i} Embedding:\n{embedding}\n")

    except Exception as e:
        print(f"Error embedding documents: {e}")

# Example usage
sample_chunks = ["This is the first chunk.", "This is the second chunk.", "And this is the third chunk."]
embed_and_view(sample_chunks)

Embedding model initialized successfully
Chunk 0 Embedding:
[0.027561187744140625, -0.030431516468524933, -0.05948604270815849, -0.043801628053188324, 0.03602953255176544, 0.005739286541938782, 0.009846016764640808, 0.040446169674396515, -0.014708821661770344, 0.004710732027888298, -0.022227749228477478, 0.07038010656833649, -0.007956956513226032, 0.00843250285834074, 0.017124926671385765, -0.0008936881786212325, -0.004870439879596233, -0.006309256888926029, -0.09675060212612152, 0.0011923924321308732, 0.021784083917737007, -0.023273255676031113, -0.022510481998324394, -0.08129142224788666, 0.012183766812086105, 0.029281053692102432, 0.05132291838526726, -0.04530042037367821, 0.06540543586015701, -0.010832075029611588, 0.03330257162451744, 0.022882545366883278, -0.008646462112665176, -0.04968956485390663, 0.003475470934063196, 0.021871499717235565, -0.04579205438494682, 0.039709657430648804, -0.009846191853284836, -0.020981719717383385, -0.05180809274315834, 0.08008384704589844, -0.050

In [ ]:
def embed_documents(text_chunks):
    """
    Function to generate embeddings for the text chunks.

    Args:
        text_chunks (list): List of text chunks from the document

    Returns:
        object: Embedding model for further use
    """
    try:
        # Initialize the Gemini embeddings
        embedding_model = GoogleGenerativeAIEmbeddings(
            model="models/text-embedding-004"  # Specify the Gemini Embedding model
        )

        print("Embedding model initialized successfully")
        return embedding_model, text_chunks
    except Exception as e:
        print(f"Error embedding documents: {e}")
        return None, None

In [ ]:
embedded_documents = embed_documents(text_chunks)

Embedding model initialized successfully


##**Section 5: Storing in Vector Database (ChromaDB)**
In this section, we'll store the embedded document chunks in a vector database for efficient semantic search.

In [ ]:
def store_embeddings(embedding_model, text_chunks):
    """
    Function to store document embeddings in ChromaDB.

    Args:
        embedding_model: The embedding model to use
        text_chunks (list): List of text chunks to embed and store

    Returns:
        object: Vector store for retrieval
    """
    try:
        # Create a vector store from the documents
        vectorstore = Chroma.from_texts(
            texts=text_chunks,
            embedding=embedding_model,
            persist_directory="./chroma_db"  # Directory to persist the database
        )

        # Persist the vector store to disk
        vectorstore.persist()

        print(f"Successfully stored {len(text_chunks)} document chunks in ChromaDB")
        return vectorstore
    except Exception as e:
        print(f"Error storing embeddings: {e}")
        return None

In [ ]:
chroma_store = store_embeddings(embedded_documents[0],embedded_documents[1])

Successfully stored 91 document chunks in ChromaDB


<ipython-input-14-ad2103a9573f>:21: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


##**Section 6: Embedding User Queries**
When a user submits a query, we need to embed it using the same embedding model to find semantically similar chunks.

In [ ]:
def embed_query(query, embedding_model):
    """
    Function to embed the user's query.

    Args:
        query (str): User's question
        embedding_model: The embedding model to use

    Returns:
        list: Embedded query vector
    """
    try:
        # Generate embedding for the query
        query_embedding = embedding_model.embed_query(query)

        print("Query embedded successfully")
        return query_embedding
    except Exception as e:
        print(f"Error embedding query: {e}")
        return None

In [ ]:
user_query = "Who are the authors of the Attention paper?"

In [ ]:
embedded_query = embed_query(user_query, embedded_documents[0])
print(embedded_query)

Query embedded successfully
[0.027056798338890076, 0.013028842397034168, -0.03088494762778282, 0.01905638352036476, -0.021859562024474144, 0.0568273700773716, 0.025342877954244614, 0.07066012918949127, 0.030687717720866203, -0.010601082816720009, -0.02538389153778553, 0.04057272896170616, 0.015543883666396141, -0.017788197845220566, -0.009052356705069542, -0.03872228413820267, 0.04835465922951698, 0.010206279344856739, 0.008470979519188404, 0.042000047862529755, -0.01562256459146738, -0.003287555417045951, -0.001992535777390003, -0.02069873735308647, 0.017305027693510056, -0.05144113674759865, 0.024168048053979874, -0.05869678035378456, -0.018119435757398605, -0.08787237107753754, -0.05156655237078667, 0.009036662988364697, -0.00023783252981957048, 0.018655333667993546, 0.0076125930063426495, 0.07520867139101028, -0.024185165762901306, 0.05185379832983017, 0.04049595445394516, -0.0520133376121521, -0.030061930418014526, -0.06000596284866333, 0.007626835256814957, 0.0522470586001873, -0

##**Section 7: Retrieval Process**
Now we'll implement the retrieval component that finds the most relevant document chunks based on the user's query.

In [ ]:
def retrieve_relevant_chunks(vectorstore, query, embedding_model, k=3):
    """
    Function to retrieve the most relevant document chunks for a query.

    Args:
        vectorstore: The ChromaDB vector store
        query (str): User's question
        embedding_model: The embedding model
        k (int): Number of chunks to retrieve

    Returns:
        list: List of relevant document chunks
    """
    try:
        # Create a retriever from the vector store
        retriever = vectorstore.as_retriever(
            search_type="similarity",  # Can also use "mmr" for Maximum Marginal Relevance
            search_kwargs={"k": k}     # Number of documents to retrieve
        )

        # Retrieve relevant chunks
        relevant_chunks = retriever.get_relevant_documents(query)

        print(f"Retrieved {len(relevant_chunks)} relevant document chunks")
        return relevant_chunks
    except Exception as e:
        print(f"Error retrieving chunks: {e}")
        return []

In [ ]:
relevant_chunks = retrieve_relevant_chunks(chroma_store, user_query, embedded_documents[0])

<ipython-input-18-d36af071108b>:22: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  relevant_chunks = retriever.get_relevant_documents(query)


Retrieved 3 relevant document chunks


In [ ]:
relevant_chunks

[Document(metadata={}, page_content='Providedproperattributionisprovided,Googleherebygrantspermissionto\nreproducethetablesandfiguresinthispapersolelyforuseinjournalisticor\nscholarlyworks.\nAttention Is All You Need\nAshishVaswani∗ NoamShazeer∗ NikiParmar∗ JakobUszkoreit∗\nGoogleBrain GoogleBrain GoogleResearch GoogleResearch\navaswani@google.com noam@google.com nikip@google.com usz@google.com\nLlionJones∗ AidanN.Gomez∗ † ŁukaszKaiser∗\nGoogleResearch UniversityofToronto GoogleBrain'),
 Document(metadata={}, page_content='attentionandtheparameter-freepositionrepresentationandbecametheotherpersoninvolvedinnearlyevery\ndetail.Nikidesigned,implemented,tunedandevaluatedcountlessmodelvariantsinouroriginalcodebaseand\ntensor2tensor.Llionalsoexperimentedwithnovelmodelvariants,wasresponsibleforourinitialcodebase,and\nefficientinferenceandvisualizations.LukaszandAidanspentcountlesslongdaysdesigningvariouspartsofand\nimplementingtensor2tensor,replacingourearliercodebase,greatlyimprovingresultsa

In [ ]:
def get_context_from_chunks(relevant_chunks, splitter="\n\n---\n\n"):
    """
    Extract page_content from document chunks and join them with a splitter.

    Args:
        relevant_chunks (list): List of document chunks from retriever
        splitter (str): String to use as separator between chunk contents

    Returns:
        str: Combined context from all chunks
    """
    # Extract page_content from each chunk
    chunk_contents = []

    for i, chunk in enumerate(relevant_chunks):
        if hasattr(chunk, 'page_content'):
            # Add a chunk identifier to help with tracing which chunk provided what information
            chunk_text = f"[Chunk {i+1}]: {chunk.page_content}"
            chunk_contents.append(chunk_text)

    # Join all contents with the splitter
    combined_context = splitter.join(chunk_contents)

    return combined_context

In [ ]:
context = get_context_from_chunks(relevant_chunks)

In [ ]:
context

'[Chunk 1]: Providedproperattributionisprovided,Googleherebygrantspermissionto\nreproducethetablesandfiguresinthispapersolelyforuseinjournalisticor\nscholarlyworks.\nAttention Is All You Need\nAshishVaswani∗ NoamShazeer∗ NikiParmar∗ JakobUszkoreit∗\nGoogleBrain GoogleBrain GoogleResearch GoogleResearch\navaswani@google.com noam@google.com nikip@google.com usz@google.com\nLlionJones∗ AidanN.Gomez∗ † ŁukaszKaiser∗\nGoogleResearch UniversityofToronto GoogleBrain\n\n---\n\n[Chunk 2]: attentionandtheparameter-freepositionrepresentationandbecametheotherpersoninvolvedinnearlyevery\ndetail.Nikidesigned,implemented,tunedandevaluatedcountlessmodelvariantsinouroriginalcodebaseand\ntensor2tensor.Llionalsoexperimentedwithnovelmodelvariants,wasresponsibleforourinitialcodebase,and\nefficientinferenceandvisualizations.LukaszandAidanspentcountlesslongdaysdesigningvariouspartsofand\nimplementingtensor2tensor,replacingourearliercodebase,greatlyimprovingresultsandmassivelyaccelerating\n\n---\n\n[Chunk 3]:

In [ ]:
 final_prompt = f"""You are a helpful assistant answering questions based on provided context.

The context is taken from academic papers, and might have formatting issues like spaces missing between words.
Please interpret the content intelligently, separating words properly when they appear joined together.

Use ONLY the following context to answer the question.
If the answer cannot be determined from the context, respond with "I cannot answer this based on the provided context."

Context:
{context}

Question: {user_query}

Answer:"""

In [ ]:
final_prompt

'You are a helpful assistant answering questions based on provided context.\n\nThe context is taken from academic papers, and might have formatting issues like spaces missing between words.\nPlease interpret the content intelligently, separating words properly when they appear joined together.\n\nUse ONLY the following context to answer the question.\nIf the answer cannot be determined from the context, respond with "I cannot answer this based on the provided context."\n\nContext:\n[Chunk 1]: Providedproperattributionisprovided,Googleherebygrantspermissionto\nreproducethetablesandfiguresinthispapersolelyforuseinjournalisticor\nscholarlyworks.\nAttention Is All You Need\nAshishVaswani∗ NoamShazeer∗ NikiParmar∗ JakobUszkoreit∗\nGoogleBrain GoogleBrain GoogleResearch GoogleResearch\navaswani@google.com noam@google.com nikip@google.com usz@google.com\nLlionJones∗ AidanN.Gomez∗ † ŁukaszKaiser∗\nGoogleResearch UniversityofToronto GoogleBrain\n\n---\n\n[Chunk 2]: attentionandtheparameter-free

In [ ]:
def generate_response(prompt, model="gemini-2.0-flash-thinking-exp-01-21", temperature=0.3, top_p=0.95):
    """
    Function to generate a response using the Gemini model.

    Args:
        prompt (str): The prompt for the model

    Returns:
        str: Model's response
    """

    llm = ChatGoogleGenerativeAI(
            model=model,
            temperature=0.2,  # Lower temperature for more focused answers
            top_p=0.95
        )

    response = llm.invoke(prompt)

    return response.content

In [ ]:
generate_response(final_prompt)

'Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Łukasz Kaiser'